In [ ]:
# Title: Fig1 ----
# Arthur BARREAU: 06 Février 2025

In [12]:
system("conda install r-sf")
system("conda install r-terra")
system("conda install r::r-ccamlrgis")  

In [ ]:
# 1. LOAD LIBRARIES ----
library(CCAMLRGIS)  
library(terra)      
library(sf)        

# 2. LOAD DATA ----
ASDs <- load_ASDs() #(Statistical Areas, Subareas, Divisions)

# 2.1 CREATE OUTPUT DIRECTORY ----
output_dir <- "cemp"
dir.create(output_dir, showWarnings = FALSE)

# 2.2 DOWNLOAD FILES ----
urls <- c(
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/cemp-shapefile-EPSG4326.shp",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/cemp-shapefile-EPSG4326.shx",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/cemp-shapefile-EPSG4326.dbf",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/cemp-shapefile-EPSG4326.prj",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/cemp-shapefile-EPSG4326.cst"
)

# Define destination file paths
destfiles <- file.path(output_dir, c(
  "cemp-shapefile-EPSG4326.shp",
  "cemp-shapefile-EPSG4326.shx",
  "cemp-shapefile-EPSG4326.dbf",
  "cemp-shapefile-EPSG4326.prj",
  "cemp-shapefile-EPSG4326.cst"
))

# Download each file and save to the output directory
for (i in seq_along(urls)) {
  download.file(urls[i], destfiles[i], mode = "wb")
}

# 2.3 READ SHAPEFILE ----
CEMP <- st_read(file.path(output_dir, "cemp-shapefile-EPSG4326.shp"))

# Reproject CEMP shapefile to match ASDs' CRS if necessary
if (st_crs(CEMP) != st_crs(ASDs)) {
  CEMP <- st_transform(CEMP, st_crs(ASDs))
}

# 3. CREATE MAP OUTPUT ----
png(filename = "outputs/Fig1.png", width = 2200, height = 1800, res = 300)

# 4. MAP PLOTTING ----
#Bathy <- load_Bathy("GEBCO2024_500.tif")
download.file("https://gis.ccamlr.org/geoserver/www/GEBCO2024_5000.tif", "GEBCO2024_5000.tif", mode = "wb")
Bathy=load_Bathy("GEBCO2024_5000.tif",Res=5000)

# Plot bathymetry with color coding for depth
plot(Bathy, breaks = Depth_cuts, col = Depth_cols, legend = FALSE, axes = FALSE, box = FALSE, mar = c(0, 0, 0, 5))

# Add a reference grid for latitude/longitude coordinates
add_RefGrid(bb = st_bbox(SmallBathy()), ResLat = 10, ResLon = 45, LabLon = 5000, fontsize = 0.75, lwd = 0.75, offset = 5)

# Add a color scale for depth visualization
add_Cscale(height = 70, fontsize = 0.75, offset = -50, width = 14, maxVal = -1, lwd = 0.5)

# Plot coastline (Antarctic land contours) in grey
plot(st_geometry(Coast[Coast$ID == 'All', ]), col = 'grey', lwd = 0.01, add = TRUE, xpd = TRUE)

# Plot ASD boundaries in red
plot(st_geometry(ASDs), add = TRUE, lwd = 0.75, border = 'red', xpd = TRUE)

# Add CEMP sites as orange points
plot(st_geometry(CEMP), add = TRUE, pch = 21, bg = 'orange', col = 'black', cex = 1.5)

# Add ASD labels for better readability
add_labels(mode = 'auto', layer = 'ASDs', fontsize = 0.6, col = 'red')

# 5. SAVE OUTPUT ----
# Close the PNG device to save the image
dev.off()

